# Charts for the website

In [1]:
cd ..


/Users/dorislee/Desktop/Research/covid/covid19-vis


In [4]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from chartlib import CovidChart, DaysSinceNumReached
import math
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
jhu_df = pd.read_csv('./data/jhu-data.csv')
# grab us-specific
jhu_df = jhu_df[(jhu_df.Country_Region == 'United States') & jhu_df.Province_State.notnull()]

# emergency declaration = e/E; restaurant closure = r/R
# border screening = b/B; travel restrictions= t/T; border closures = c/C
# shelter-in-place = l/L; gathering limitations= g/G; k-12 school closures = s/S
# non-essential business closure = n/N
def create_lockdown_type (x, emo_flag):
    s = r = emo_s = ""
    regional_flag = closure_flag = not_the_first_flag = 0
    if (x['Coverage'] != 'State-wide'):
        r = 'Regional'
        regional_flag = 1
    if (x['State of Emergency Declaration'] == "State of Emergency declared"):
        s = s + " Declaration of Emergency"
        not_the_first_flag = 1
        emo_s = (emo_s + "e") if (regional_flag == 1) else (emo_s + "E")
    if (x['Travel Restrictions'] == x['Travel Restrictions']):
        if (not_the_first_flag == 1): s = s + ","
        if ("Travel restrictions for out of state travelers" in x['Travel Restrictions']):
            emo_s = (emo_s + "t") if (regional_flag == 1) else (emo_s + "T")
        if ("Border closures" in x['Travel Restrictions']):
            emo_s = (emo_s + "c") if (regional_flag == 1) else (emo_s + "C")
        s = s + " Border Closure/Visitor Quarantine"
        not_the_first_flag = 1
    if (x['Shelter-in-place Order'] == x['Shelter-in-place Order']):
        if (not_the_first_flag == 1): s = s + ","
        if (x['Shelter-in-place Order'] == "Shelter-in-place order"): 
            s = s + " Stay-at-home Order"
        if (x['Shelter-in-place Order'] == "Night-time curfew"):
            s = s + " Curfew"
        emo_s = (emo_s + "l") if (regional_flag == 1) else (emo_s + "L")
        not_the_first_flag = 1
    if (x['Banning Gatherings of a Certain Size'] == x['Banning Gatherings of a Certain Size']):
        if (not_the_first_flag == 1): s = s + ","
        s = s + " Gatherings (>"+str(int(x['Banning Gatherings of a Certain Size']))+") Banned"
        emo_s = (emo_s + "g") if (regional_flag == 1) else (emo_s + "G")
        not_the_first_flag = 1
    if (x['K-12 School Closure'] == x['K-12 School Closure']):
        if (not_the_first_flag == 1): s = s + ","
        s = s + " Closure of Schools"
        emo_s = (emo_s + "s") if (regional_flag == 1) else (emo_s + "S")
        closure_flag = not_the_first_flag = 1
    if (x['Bar and Dine-in Restaurant Closure'] == x['Bar and Dine-in Restaurant Closure']):
        if (not_the_first_flag == 1): s = s + ","
        s = (s + " Closure of Restaurants") if (closure_flag == 0) else (s + " Restaurants")
        emo_s = (emo_s + "r") if (regional_flag == 1) else (emo_s + "R")
        closure_flag = not_the_first_flag = 1
    if (x['Non-essential Businesses Closure'] == x['Non-essential Businesses Closure']):
        if (not_the_first_flag == 1): s = s + ","
        s = (s + " Closure of Non-essential Businesses") if (closure_flag == 0) else (s + " Non-essential Businesses")
        emo_s = (emo_s + "n") if (regional_flag == 1) else (emo_s + "N")
        closure_flag = not_the_first_flag = 1
    if (emo_flag == 1): return emo_s;
    if (s == ""):
        return s
    return (r + s).strip()


quarantine_df = pd.read_csv('./data/quarantine-activity-US-Apr16.csv')
quarantine_df = quarantine_df.rename(columns={'State': 'Province_State', 'Effective Date': 'lockdown_date'})
quarantine_df['lockdown_type'] = quarantine_df.apply(lambda x: create_lockdown_type(x, 0), axis=1)
quarantine_df['emoji_string']  = quarantine_df.apply(lambda x: create_lockdown_type(x, 1), axis=1)
quarantine_df['lockdown_type'].replace('', np.nan, inplace=True)
quarantine_df = quarantine_df.dropna(subset = ['lockdown_type'])
quarantine_df = quarantine_df.groupby(['lockdown_date', 'Province_State']).agg({'lockdown_type': lambda col: '; '.join(col), 'emoji_string':lambda col: ''.join(col)}).reset_index()
quarantine_cols = ['Province_State', 'lockdown_date', 'lockdown_type', 'emoji_string']
quarantine_df = quarantine_df[quarantine_cols]

#quarantine_df = quarantine_df.loc[quarantine_df.Province_State== 'Florida']
quarantine_df



days_since = 20
chart = CovidChart(
    jhu_df,
    groupcol='Province_State',
    start_criterion=DaysSinceNumReached(days_since, 'Confirmed'),
    ycol='Confirmed',
    level='USA',
    xcol='Date',
    top_k_groups=10,
    quarantine_df=quarantine_df  # should have a column with same name as `groupcol`
)
# chart.set_colormap()
chart.set_unfocused_opacity(0.05)
chart = chart.set_ytitle('Num Confirmed Cases (log)')
chart = chart.set_xtitle('Days since {} Confirmed'.format(days_since))
chart.set_width(800).set_height(400)
chart.set_xdomain((0, 35)).set_ydomain((days_since, 100000))
# chart.set_title("States With Significant Rate Decreases")
# chart.facetby("Significant Decrease")
# chart.spec.column = "Significant Decrease"
chart.compile()

alt.LayerChart(...)

In [6]:
import numpy as np
jhu_df = pd.read_csv('./data/jhu-data.csv')
jhu_df = jhu_df[(jhu_df.Province_State.isnull()) & (jhu_df.Country_Region != 'China')]

# border screening = b/B; travel restrictions= t/T; border closures = c/C
# shelter-in-place = l/L; gathering limitations= g/G; k-12 school closures = s/S
# non-essential business closure = n/N
def create_lockdown_type_world (x, emo_flag):
    s = r = emo_s = ""
    regional_flag = closure_flag = not_the_first_flag = 0 
    if (x['coverage'] == 'Targeted'):
        r = 'Regional'
        regional_flag = 1
    if (x['Travel Restrictions'] == "Screening"): # assumption: only one travel restriction at a time
        s = s + " Border Screening"
        not_the_first_flag = 1
        emo_s = (emo_s + "b") if (regional_flag == 1) else (emo_s + "B")
    elif (x['Travel Restrictions'] == "Quarantine on high-risk regions"):
        s = s + " Visitor Quarantine"
        not_the_first_flag = 1
        emo_s = (emo_s + "t") if (regional_flag == 1) else (emo_s + "T")
    elif (x['Travel Restrictions'] == "Ban on high risk regions"):
        s = s + " Border Closures"
        not_the_first_flag = 1
        emo_s = (emo_s + "c") if (regional_flag == 1) else (emo_s + "C")
    if (x['Shelter-in-place Order'] == "Restrict movement"):
        # Movement restriction recommended is omitted
        if (not_the_first_flag == 1): s = s + ","
        s = s + " Stay-at-home Order"
        not_the_first_flag = 1
        emo_s = (emo_s + "l") if (regional_flag == 1) else (emo_s + "L")
    if (x['Gathering Limitations'] == "Required Cancelling Public Events"):
        # Recommend Cancelling Public Events is omitted
        if (not_the_first_flag == 1): s = s + ","
        s = s + " Gatherings Banned"
        not_the_first_flag = 1
        emo_s = (emo_s + "g") if (regional_flag == 1) else (emo_s +"G")
    if (x['K-12 School Closure'] == "Required Closing"):
        if (not_the_first_flag == 1):
            s = s + ","
        s = s + " Closure of Schools"
        closure_flag = 1
        not_the_first_flag = 1
        emo_s = (emo_s + "s") if (regional_flag == 1) else (emo_s + "S")
    if (x['Non-essential Businesses Closure'] == "Required Closing Workspaces"):
        # Recommend Closing Workspaces is omitted
        if (not_the_first_flag == 1):
            s = s + ","
        if (closure_flag == 0):
            s = s + " Closure of Non-essential Businesses"
        else:
            s = s + " Non-essential Businesses"
        closure_flag = 1
        not_the_first_flag = 1
        emo_s = (emo_s + "n") if (regional_flag == 1) else (emo_s + "N")
    if (emo_flag == 1): return emo_s
    if (s == ""): return s # Ensures just "Regional" is not returned
    return (r + s).strip()

def strip_nans(x):
    # gets rid of nans in a list
    if (type(x) == list):
        x_strip = ""
        for a in x:
            if (a == a):
                # not nan
                x_strip = x_strip + a.strip()
        return x_strip
    else:
        return x.strip()


quarantine_df = pd.read_csv ('./data/quarantine-activity-Apr19.csv')
quarantine_df = quarantine_df = quarantine_df.rename(
     columns  = {'date': 'lockdown_date', 'country_name': 'Country_Region'}
)
#quarantine_df = quarantine_df.loc[quarantine_df.Country_Region == 'Turkey']


quarantine_df = quarantine_df.groupby(['lockdown_date', 'Country_Region', 'coverage']).agg({'Travel Restrictions': list, 'Gathering Limitations': list,'Shelter-in-place Order':list, 'K-12 School Closure':list,'Non-essential Businesses Closure':list}).reset_index()
quarantine_df = quarantine_df.applymap(strip_nans)
quarantine_df['lockdown_type'] = quarantine_df.apply(lambda x: create_lockdown_type_world(x, 0), axis=1)
quarantine_df['emoji_string']  = quarantine_df.apply(lambda x: create_lockdown_type_world(x, 1), axis=1)
quarantine_df['lockdown_type'].replace('', np.nan, inplace=True)
quarantine_df = quarantine_df.dropna(subset = ['lockdown_type'])
quarantine_df = quarantine_df.groupby(['lockdown_date', 'Country_Region']).agg({'lockdown_type': lambda col: '; '.join(col), 'emoji_string':lambda col: ''.join(col)}).reset_index()
quarantine_cols = ['Country_Region', 'lockdown_date', 'lockdown_type', 'emoji_string']
quarantine_df = quarantine_df[quarantine_cols]
quarantine_df


days_since = 50
chart = CovidChart(
    jhu_df,
    groupcol='Country_Region',
    start_criterion=DaysSinceNumReached(days_since, 'Confirmed'),
    ycol='Confirmed',
    level='country',
    xcol='Date',
    top_k_groups=30,
    quarantine_df=quarantine_df
)

# chart.set_colormap()
chart.set_unfocused_opacity(0.05)
chart = chart.set_ytitle('Number of Confirmed Cases (log)')
chart = chart.set_xtitle('Days since {} Confirmed'.format(days_since))
chart.set_width(600).set_height(400)
chart.set_ydomain((days_since, 200000))
chart.set_xdomain((0, 40))
chart.compile()

alt.LayerChart(...)

In [ ]:
#jhu_df.loc[(jhu_df["Province_State"]=="Illinois")|(jhu_df["Province_State"]=="New York")| (jhu_df["Province_State"]=="New Jersey")| (jhu_df["Province_State"]=="California") | (jhu_df["Province_State"]=="Washington")| (jhu_df["Province_State"]=="Michigan")| (jhu_df["Province_State"]=="Louisiana"),"Significant Decrease"] = True
#jhu_df.loc[(jhu_df["Province_State"]=="Massachusetts")|(jhu_df["Province_State"]=="Florida")| (jhu_df["Province_State"]=="Texas")| (jhu_df["Province_State"]=="Pennsylvania"),"Significant Decrease"]=False
#jhu_df = jhu_df[~jhu_df["Significant Decrease"].isna()]

In [ ]:
# rename SK
# quarantine_df = quarantine_df.loc[quarantine_df.Level == 'Enforcement']
# quarantine_df = quarantine_df.loc[quarantine_df.Type != 'Border Control']
# quarantine_df['Lockdown Type'] = quarantine_df.apply(
#         lambda x: x['Scope'] + ' ' + x['Type'], axis=1
# )
# quarantine_cols = ['Country_Region', 'Date Enacted', 'Lockdown Type']
# quarantine_df = quarantine_df[quarantine_cols]
# quarantine_df = quarantine_df.rename(
#     columns={'Date Enacted': 'lockdown_date', 'Lockdown Type': 'lockdown_type'}
# )
# quarantine_df.loc[
#     quarantine_df.lockdown_type == 'Partial Internal Lockdown', 'lockdown_type'
# ] = 'Region-Specific Countermeasures Begin'


In [ ]:
# Province_State	lockdown_date	lockdown_type
# quarantine_df = quarantine_df.loc[quarantine_df.Regions == 'All']
# quarantine_df_emergency = quarantine_df.copy()
# quarantine_df = quarantine_df.loc[(quarantine_df.Type == 'Level 2 Lockdown') | (quarantine_df.Type == 'Level 1 Lockdown')]
# quarantine_df['Lockdown Type'] = 'Full Lockdown'
# quarantine_cols = ['Province_State', 'Date Enacted', 'Lockdown Type']
# quarantine_df = quarantine_df[quarantine_cols]
# quarantine_df_emergency['Lockdown Type'] = 'Emergency Declared'
# quarantine_cols_emergency = ['Province_State', 'State of emergency declared', 'Lockdown Type']
# quarantine_df_emergency = quarantine_df_emergency[quarantine_cols_emergency]
# quarantine_df_emergency = quarantine_df_emergency.rename(columns={'State of emergency declared': 'Date Enacted'})
# quarantine_df = pd.concat([quarantine_df, quarantine_df_emergency])
# quarantine_df = quarantine_df.rename(
#     columns={'Date Enacted': 'lockdown_date', 'Lockdown Type': 'lockdown_type'}
# )

# quarantine_df=pd.read_csv('./data/quarantine-activity-US.csv')
# quarantine_df

# days_since = 20
# chart = CovidChart(
#     jhu_df,
#     groupcol='Province_State',
#     start_criterion=DaysSinceNumReached(days_since, 'Confirmed'),
#     ycol='Confirmed',
#     level='USA',
#     xcol='Date',
#     top_k_groups=10,
#     quarantine_df='./data/quarantine-activity-US.csv'  # should have a column with same name as `groupcol`
# )
# # chart.set_colormap()
# chart.set_unfocused_opacity(0.05)
# chart = chart.set_ytitle('Num Confirmed Cases (log)')
# chart = chart.set_xtitle('Days since {} Confirmed'.format(days_since))
# chart.set_width(800).set_height(400)
# chart.set_xdomain((0, 35)).set_ydomain((days_since, 100000))
# # chart.set_title("States With Significant Rate Decreases")
# # chart.facetby("Significant Decrease")
# # chart.spec.column = "Significant Decrease"
# chart.compile()

In [7]:
# Example 3: JHU World

jhu_df = pd.read_csv("./data/jhu-data.csv")
jhu_df = jhu_df[jhu_df["Province_State"].isnull()]

quarantine_df = pd.read_csv('./data/quarantine-activity.csv')
quarantine_cols = ['Country_Region', 'Date Enacted', 'Scope']
quarantine_df
#quarantine_df = quarantine_df[quarantine_cols]


# # CovidChart expects to have the following columns
# quarantine_df = quarantine_df.rename(columns={'Date Enacted': 'lockdown_date', 'Lockdown Type': 'lockdown_type'})
# # quarantine_df = quarantine_df.loc[quarantine_df.lockdown_type == 'Full']


# chart = CovidChart(
#     jhu_df, 
#     groupcol='Country_Region', 
#     start_criterion=DaysSinceNumReached(50, 'Confirmed'), 
#     ycol='Confirmed',
#     xcol='Date',
#     top_k_groups=10,
#     quarantine_df=quarantine_df # should have a column with same name as `groupcol`
# )

# chart = chart.set_ytitle('Num Confirmed Cases (log)')
# chart = chart.set_xtitle('Days since 50 Confirmed')
# spec = chart.spec
# # note: spec object autocompletes
# # can use it to adjust chart properties

# # spec.lockdown_extrapolation = False
# # spec.yscale = 'linear'
# # spec.points = False
# # spec.tooltip_rules = False
# chart.set_width(500).set_height(400)
# chart.set_ydomain((100, 200000))
# jhu_altChart = chart.set_xdomain((0, 40)).compile()
# jhu_altChart

,Display,Country_ID,Country_Region,Date Enacted,Planned end date,Actual end date,Type,Scope,Level,(Deprecated) Level,(Deprecated) Lockdown Type,Comments,Reference,Last updated
0,1,AF,Afghanistan,03-25-2020,NaN,NaN,Internal Lockdown,Partial,Enforcement,Regional,Partial,Day-time curfew in three provinces,https://www.aa.com.tr/en/asia-pacific/covid-19...,03-25-2020
1,1,AF,Afghanistan,03-27-2020,NaN,NaN,Internal Lockdown,Partial,Enforcement,Regional,Partial,Full lockdown in Kabul,https://en.wikipedia.org/wiki/2020_coronavirus...,04-04-2020
2,1,AL,Albania,03-10-2020,NaN,NaN,Internal Lockdown,Full,Enforcement,National,Full,NaN,https://en.wikipedia.org/wiki/2020_coronavirus...,03-25-2020
3,1,DZ,Algeria,03-24-2020,04-11-2020,NaN,Internal Lockdown,Partial,Enforcement,Regional,Partial,"Full lockdown for the Wilayat of Blida, partia...",https://www.garda.com/crisis24/news-alerts/325...,03-25-2020
4,1,AD,Andorra,03-16-2020,NaN,NaN,Internal Lockdown,Full,Enforcement,National,Full,NaN,https://www.independent.co.uk/news/world/europ...,03-27-2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,0,UY,Uruguay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273,0,UZ,Uzbekistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274,0,VU,Vanuatu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275,0,ZM,Zambia,NaN,NaN,NaN,Border Control,NaN,None,NaN,None,NaN,NaN,03-27-2020


In [75]:
chart.export("../website/jhu_country.js", "jhu_country")

In [26]:
chart.set_width(300).set_height(400)
jhu_altChart = chart.set_xdomain((0, 40)).compile()

#drop countries that do not have a lockdown date
jhu_altChart.data = jhu_altChart.data[~jhu_altChart.data.lockdown_date.isnull()]

jhu_altChart.data["early_late"] = "late"
jhu_altChart.data.loc[jhu_altChart.data["lockdown_x"]<10,"early_late"] = "early"

jhu_altChart.facet(column="early_late")

#TODO: There is a bug here where the prediction line is only showing up for the right chart and not the left chart

alt.FacetChart(...)

In [27]:
jhu_altChart.facet(column="lockdown_type")

alt.FacetChart(...)

In [76]:
quarantine_df = pd.read_csv('./data/quarantine-activity-US.csv')
quarantine_cols = ['Province_State', 'Date Enacted', 'Lockdown Type']
quarantine_df = quarantine_df[quarantine_cols]

# CovidChart expects to have the following columns
quarantine_df = quarantine_df.rename(columns={'Date Enacted': 'lockdown_date', 'Lockdown Type': 'lockdown_type'})

# needs to have name identical to groupcol
nyt_quarantine_df = quarantine_df.rename(columns={'Province_State': 'state'})
nyt_chart = CovidChart(
    './nyt-us-states.csv', 
    groupcol='state', 
    start_criterion=DaysSinceNumReached(10, 'cases'), 
    ycol='cases', 
    top_k_groups=10, 
    quarantine_df=nyt_quarantine_df
)
nyt_chart = nyt_chart.set_ytitle('Confirmed (log)')
nyt_chart = nyt_chart.set_xtitle('Days since 10 Cases')
spec = nyt_chart.spec
spec.lines = True
# spec.points = False
# spec.tooltip_text = False
# spec.tooltip_rules = False
# spec.tooltip_points = False
# spec.has_tooltips = False

spec.lockdown_extrapolation = True
spec.tooltip_text = True
nyt_chart.set_width(600).set_height(400)
nyt_chart.set_xdomain((0, 35)).set_ydomain((10, 100000))
altChart = nyt_chart.compile()
altChart

alt.LayerChart(...)

In [ ]:
nyt_chart.export("jhu_country.js", "jhu_country")

In [30]:
nyt_chart.set_width(300).set_height(400)
altChart = nyt_chart.set_xdomain((0, 40)).compile()

#drop countries that do not have a lockdown date
altChart.data = altChart.data[~altChart.data.lockdown_date.isnull()]

altChart.data["early_late"] = "late"
altChart.data.loc[altChart.data["lockdown_x"]<15,"early_late"] = "early"

altChart.facet(column="early_late")


alt.FacetChart(...)

In [31]:
altChart.facet(column="lockdown_type")

alt.FacetChart(...)

In [ ]:
altChart.facet(column="early_late",row="lockdown_type")

In [32]:
altChart.facet(column="early_late",row="lockdown_type")

alt.FacetChart(...)

### Selected Examples

In [68]:
nyt_df = pd.read_csv('./nyt-us-states.csv')
filtered_nyt_df = nyt_df[(nyt_df["state"]=="Illinois")|(nyt_df["state"]=="New York")| (nyt_df["state"]=="New Jersey")| (nyt_df["state"]=="Washington")| (nyt_df["state"]=="Michigan")]
nyt_chart = CovidChart(
    filtered_nyt_df, 
    groupcol='state', 
    start_criterion=DaysSinceNumReached(10, 'cases'), 
    ycol='cases', 
    top_k_groups=20, 
    quarantine_df=nyt_quarantine_df
)
nyt_chart = nyt_chart.set_ytitle('Confirmed (log)')
nyt_chart = nyt_chart.set_xtitle('Days since 10 Cases')
spec = nyt_chart.spec
spec.lines = True

spec.lockdown_extrapolation = True
spec.tooltip_text = True
nyt_chart.set_width(250).set_height(400)
nyt_chart.set_xdomain((0, 35)).set_ydomain((10, 100000))
nyt_chart.set_title("States With Significant Rate Decreases")
altChart = nyt_chart.compile()
altChart

alt.LayerChart(...)

In [69]:
nyt_chart.export("../website/state_big_changes.js", "state_big_changes")

In [70]:
nyt_df = pd.read_csv('./nyt-us-states.csv')
filtered_nyt_df = nyt_df[(nyt_df["state"]=="Massachusetts")|(nyt_df["state"]=="Florida")| (nyt_df["state"]=="Texas")| (nyt_df["state"]=="California")| (nyt_df["state"]=="Pennsylvania")]
nyt_chart = CovidChart(
    filtered_nyt_df, 
    groupcol='state', 
    start_criterion=DaysSinceNumReached(10, 'cases'), 
    ycol='cases', 
    top_k_groups=20, 
    quarantine_df=nyt_quarantine_df
)
nyt_chart = nyt_chart.set_ytitle('Confirmed (log)')
nyt_chart = nyt_chart.set_xtitle('Days since 10 Cases')
spec = nyt_chart.spec
spec.lines = True

spec.lockdown_extrapolation = True
spec.tooltip_text = True
nyt_chart.set_width(250).set_height(400)
nyt_chart.set_xdomain((0, 35)).set_ydomain((10, 100000))
nyt_chart.set_title( "States Without Significant Rate Change")

altChart = nyt_chart.compile()
altChart

alt.LayerChart(...)

In [71]:
nyt_chart.export("../website/state_minimal_changes.js", "state_minimal_changes")

### Death

In [140]:
# Example 3: JHU World

jhu_df = pd.read_csv("./jhu-data.csv")
jhu_df = jhu_df[jhu_df["Province_State"].isnull()]

quarantine_df = pd.read_csv('quarantine-activity.csv')
quarantine_cols = ['Country_Region', 'Date Enacted', 'Lockdown Type']
quarantine_df = quarantine_df[quarantine_cols]


# CovidChart expects to have the following columns
quarantine_df = quarantine_df.rename(columns={'Date Enacted': 'lockdown_date', 'Lockdown Type': 'lockdown_type'})
# quarantine_df = quarantine_df.loc[quarantine_df.lockdown_type == 'Full']


chart = CovidChart(
    jhu_df, 
    groupcol='Country_Region', 
    start_criterion=DaysSinceNumReached(10, 'Deaths'), 
    ycol='Deaths',
    xcol='Date',
    top_k_groups=10,
    quarantine_df=quarantine_df # should have a column with same name as `groupcol`
)

chart = chart.set_ytitle('Number of Deaths (log)')
chart = chart.set_xtitle('Days since 10 Deaths')
spec = chart.spec
# note: spec object autocompletes
# can use it to adjust chart properties

# spec.lockdown_extrapolation = False
# spec.yscale = 'linear'
# spec.points = False
# spec.tooltip_rules = False
chart.set_width(500).set_height(400)
chart.set_ydomain((10, 10000))
altChart = chart.set_xdomain((0, 35)).compile()

chart.export("../website/deaths_by_country.js", "deaths_by_country")
altChart

alt.LayerChart(...)

In [144]:
quarantine_df = pd.read_csv('quarantine-activity-US.csv')
quarantine_cols = ['Province_State', 'Date Enacted', 'Lockdown Type']
quarantine_df = quarantine_df[quarantine_cols]

# CovidChart expects to have the following columns
quarantine_df = quarantine_df.rename(columns={'Date Enacted': 'lockdown_date', 'Lockdown Type': 'lockdown_type'})

# needs to have name identical to groupcol
nyt_quarantine_df = quarantine_df.rename(columns={'Province_State': 'state'})
nyt_chart = CovidChart(
    './nyt-us-states.csv', 
    groupcol='state', 
    start_criterion=DaysSinceNumReached(10, 'deaths'), 
    ycol='deaths', 
    top_k_groups=10, 
    quarantine_df=nyt_quarantine_df
)
nyt_chart = nyt_chart.set_ytitle('Deaths (log)')
nyt_chart = nyt_chart.set_xtitle('Days since 10 Cases')
spec = nyt_chart.spec
spec.lines = True
# spec.points = False
# spec.tooltip_text = False
# spec.tooltip_rules = False
# spec.tooltip_points = False
# spec.has_tooltips = False

spec.lockdown_extrapolation = True
spec.tooltip_text = True
nyt_chart.set_width(500).set_height(400)
nyt_chart.set_xdomain((0, 20)).set_ydomain((10, 500))
altChart = nyt_chart.compile()
nyt_chart.export("../website/deaths_by_state.js", "deaths_by_state")
altChart

alt.LayerChart(...)

### Rate of Change

In [145]:
#trying double derivative
def computeAvrgDailyChange(x,window=7):
    pctChange = x.diff()/x*100
    return pctChange.rolling(window=window).mean()
quarantine_df = pd.read_csv('quarantine-activity-US.csv')
quarantine_cols = ['Province_State', 'Date Enacted', 'Lockdown Type']
quarantine_df = quarantine_df[quarantine_cols]
# CovidChart expects to have the following columns
quarantine_df = quarantine_df.rename(columns={'Date Enacted': 'lockdown_date', 'Lockdown Type': 'lockdown_type'})
# needs to have name identical to groupcol
nyt_quarantine_df = quarantine_df.rename(columns={'Province_State': 'state'})
nyt_df = pd.read_csv('./nyt-us-states.csv')
nyt_df["cases"] = nyt_df.groupby("state")["cases"].transform(computeAvrgDailyChange)
nyt_chart = CovidChart(
    nyt_df, 
    groupcol='state', 
    start_criterion=DaysSinceNumReached(10, 'cases'), 
    ycol='cases', 
    top_k_groups=10, 
    quarantine_df=nyt_quarantine_df
)
nyt_chart = nyt_chart.set_ytitle('Average Daily Change in Confirmed Cases')
nyt_chart = nyt_chart.set_xtitle('Days since 10 Cases')
spec = nyt_chart.spec
spec.lines = True
# spec.tooltip_text = False
# spec.tooltip_rules = False
# spec.tooltip_points = False
spec.yscale = 'linear'
spec.has_tooltips = False
spec.points = False
spec.lockdown_extrapolation = False
spec.tooltip_text = True
nyt_chart.set_width(500).set_height(400)
nyt_chart.set_xdomain((0, 25)).set_ydomain((10, 50))


nyt_chart.export("../website/change_by_state.js", "change_by_state")
nyt_chart.compile()

alt.LayerChart(...)

In [143]:
jhu_df = pd.read_csv("./jhu-data.csv")
jhu_df = jhu_df[jhu_df["Province_State"].isnull()]
jhu_df["Average Daily Change in Confirmed Cases"] = jhu_df.groupby("Country_Region")["Confirmed"].transform(computeAvrgDailyChange)

quarantine_df = pd.read_csv('quarantine-activity.csv')
quarantine_cols = ['Country_Region', 'Date Enacted', 'Lockdown Type']
quarantine_df = quarantine_df[quarantine_cols]


# CovidChart expects to have the following columns
quarantine_df = quarantine_df.rename(columns={'Date Enacted': 'lockdown_date', 'Lockdown Type': 'lockdown_type'})
# quarantine_df = quarantine_df.loc[quarantine_df.lockdown_type == 'Full']


chart = CovidChart(
    jhu_df, 
    groupcol='Country_Region', 
    start_criterion=DaysSinceNumReached(10, "Confirmed"), 
    ycol="Average Daily Change in Confirmed Cases",
    xcol='Date',
    top_k_groups=10, # Problem: top k is based on ycol but not confirmed cases
    quarantine_df=quarantine_df # should have a column with same name as `groupcol`
)

chart = chart.set_ytitle("Average Daily Change in Confirmed Cases")
chart = chart.set_xtitle('Days since 10 Confirmed')
spec = chart.spec
# note: spec object autocompletes
# can use it to adjust chart properties

# spec.lockdown_extrapolation = False
spec.yscale = 'linear'
# spec.points = False
# spec.tooltip_rules = False
chart.set_width(500).set_xdomain((0, 25)).set_ydomain((10, 40))
altChart = chart.set_xdomain((0, 35)).compile()

chart.export("../website/change_by_country.js", "change_by_country")
altChart

alt.LayerChart(...)